In [21]:
# src/option_greeks.py
import numpy as np
from scipy.stats import norm

def d1(S, K, T, r, sigma, q=0.0):
    """Helper function: d1 term in Black-Scholes."""
    return (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def d2(S, K, T, r, sigma, q=0.0):
    """Helper function: d2 term in Black-Scholes."""
    return d1(S, K, T, r, sigma, q) - sigma * np.sqrt(T)

# -----------------------
# Greeks Implementation
# -----------------------

def delta(S, K, T, r, sigma, option_type="call", q=0.0):
    """Delta: sensitivity to underlying price."""
    d1_val = d1(S, K, T, r, sigma, q)
    if option_type.lower() == "call":
        return np.exp(-q * T) * norm.cdf(d1_val)
    else:  # put
        return -np.exp(-q * T) * norm.cdf(-d1_val)

def gamma(S, K, T, r, sigma, q=0.0):
    """Gamma: sensitivity of Delta to underlying price."""
    d1_val = d1(S, K, T, r, sigma, q)
    return np.exp(-q * T) * norm.pdf(d1_val) / (S * sigma * np.sqrt(T))

def vega(S, K, T, r, sigma, q=0.0):
    """Vega: sensitivity to volatility (per 1% change)."""
    d1_val = d1(S, K, T, r, sigma, q)
    return S * np.exp(-q * T) * norm.pdf(d1_val) * np.sqrt(T) / 100.0

def theta(S, K, T, r, sigma, option_type="call", q=0.0):
    """Theta: sensitivity to time (per day)."""
    d1_val = d1(S, K, T, r, sigma, q)
    d2_val = d2(S, K, T, r, sigma, q)
    first_term = -(S * np.exp(-q * T) * norm.pdf(d1_val) * sigma) / (2 * np.sqrt(T))

    if option_type.lower() == "call":
        second_term = q * S * np.exp(-q * T) * norm.cdf(d1_val)
        third_term = -r * K * np.exp(-r * T) * norm.cdf(d2_val)
        theta_val = first_term + second_term + third_term
    else:  # put
        second_term = -q * S * np.exp(-q * T) * norm.cdf(-d1_val)
        third_term = r * K * np.exp(-r * T) * norm.cdf(-d2_val)
        theta_val = first_term + second_term + third_term

    return theta_val / 365.0  # per day

def rho(S, K, T, r, sigma, option_type="call", q=0.0):
    """Rho: sensitivity to interest rate (per 1% change)."""
    d2_val = d2(S, K, T, r, sigma, q)
    if option_type.lower() == "call":
        return K * T * np.exp(-r * T) * norm.cdf(d2_val) / 100.0
    else:  # put
        return -K * T * np.exp(-r * T) * norm.cdf(-d2_val) / 100.0

# -----------------------
# Example Run
# -----------------------
if __name__ == "__main__":
    # Example parameters
    S = 100      # Spot price
    K = 100      # Strike price
    T = 1        # Time to maturity (1 year)
    r = 0.05     # Risk-free rate
    sigma = 0.2  # Volatility
    q = 0.0      # Dividend yield

    for opt in ["call", "put"]:
        print(f"\n{opt.upper()} option Greeks:")
        print("Delta:", delta(S, K, T, r, sigma, opt, q))
        print("Gamma:", gamma(S, K, T, r, sigma, q))
        print("Vega :", vega(S, K, T, r, sigma, q))
        print("Theta:", theta(S, K, T, r, sigma, opt, q))
        print("Rho  :", rho(S, K, T, r, sigma, opt, q))



CALL option Greeks:
Delta: 0.6368306511756191
Gamma: 0.018762017345846895
Vega : 0.3752403469169379
Theta: -0.01757267820941972
Rho  : 0.5323248154537634

PUT option Greeks:
Delta: -0.3631693488243809
Gamma: 0.018762017345846895
Vega : 0.3752403469169379
Theta: -0.004542138147766099
Rho  : -0.4189046090469506
